# Learn And implement Pytorch Part - 1
---

## Kaggle competitions

![alt text](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/02/pytorch-logo-flat-300x210.png)

#### Important Links
* https://towardsdatascience.com/a-beginners-tutorial-on-building-an-ai-image-classifier-using-pytorch-6f85cb69cba7

* https://www.kaggle.com/carloalbertobarbano/vgg16-transfer-learning-pytorch
* http://scikit-image.org/docs/dev/user_guide/numpy_images.html


*By Pankaj Pundir*

# google drive access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/My Drive/Colab Notebooks/pyTorch")

# installing pytorch and torchvision
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision


# Import **Libraries**

In [1]:
# basic imports of pyTorch
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# transfer learning model import
from torchvision import models

from torch.autograd import Variable
import torch.nn.functional as F
print(torch.cuda.is_available())

# some sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


# basic python imports
import cv2, os , sys
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
%matplotlib inline

False


/home/alvin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/alvin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/alvin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/alvin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/alvin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## torch information

In [ ]:
print(torch.__version__)

print(torch.cuda.get_device_name(0))

# print device properties 
print(torch.cuda.get_device_properties(0))


# empty the cache of cuda
torch.cuda.empty_cache()

# Data  Loading - and accessibility 
Robust Class method **bold text**



*   provide Train & test option.
*   The data is provided as generator thus saves memory.
* preprocessing is adjustable with ** **bold text** cv2**





In [ ]:
class CustomedDataSet(torch.utils.data.Dataset):
  
    def __init__(self, train=True):
        self.train = train
        self.train_loc = '../input/train.csv'
        self.test_loc = '../input/test.csv'
        
        if self.train :
            trainX = pd.read_csv(self.train_loc)
            
            trainY = trainX.label.values.tolist()
            
            trainX = np.apply_along_axis(self.preprocess , axis=1 ,\
                  arr = trainX.drop('label',axis=1).values.reshape(trainX.shape[0], 1, 28, 28).astype(np.uint8))
            
            # storing data to the list
            self.datalist = trainX
            self.labellist = trainY
       
        else:
            testX = pd.read_csv(self.test_loc)
            testX = testX.values.reshape(testX.shape[0], 1, 28, 28)
            self.datalist = testX
            
    def __getitem__(self, index):
        # train - provide Xtrain and Ytrain
        
        if self.train:
            return torch.Tensor(self.datalist[index].astype(float)), self.labellist[index]
        else:
            return torch.Tensor(self.datalist[index].astype(float))
    
    def __len__(self):
        return self.datalist.shape[0]
    
    def preprocess(self,img):
        '''
        Apply transformation that are not implicitly provided in the
        pytorch inbuilt functions.
        '''
        
        print(img.shape)
        blur = cv2.GaussianBlur(img,(3,3),0)
        ret3,th3 = cv2.threshold(blur,120,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return img
      
      
   
        
# should be initialized once -or delete previos instance of 
# object as increased in size and RAM consumption

train_dataset = CustomedDataSet()
test_dataset = CustomedDataSet(train=False)

sys.getsizeof(train_dataset)

### Updated functionality of Custom data

In [ ]:
   
class CustomedDataSet(torch.utils.data.Dataset):
    '''
    CustomedDataSet provide the data to be converted to
    train, validation and test set.
    num_rows = (1000,1500) - select the set of rows
    train = bool - composes of train and validation
    
    '''
    def __init__(self, train=True, num_rows = (0,10) ):
        self.train = train
        
        self.datalist =[]
        
        self.transform = transforms.Compose([
        np.uint8,
        transforms.ToPILImage(),
        transforms.Resize(size=(224,224), interpolation = 2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
        ])
        
        
        
        if self.train :
            trainX = pd.read_csv('../input/train.csv',nrows = num_rows[1])
            trainX = trainX.iloc[num_rows[0]:,:]
            trainY = trainX.label.values.tolist()
            arr = trainX.drop('label',axis=1).values.reshape(trainX.shape[0],  28, 28).astype(np.uint8)
            for i in arr:
                self.datalist.append(self.preprocess(i))
        
            self.datalist = torch.stack(self.datalist)
            self.labellist = trainY
            if num_rows[0] != 0:
                print(f"Validation Data : {trainX.shape}")
            else:
                print(f"Training Data : {trainX.shape}")
            
        else:
            testX = pd.read_csv('../input/test.csv', nrows = num_rows[1])
            testX = testX.values.reshape(testX.shape[0], 28, 28).astype(np.uint8)
            for i in testX:
                self.datalist.append(self.preprocess(i))
        
            self.datalist = torch.stack(self.datalist)
#             self.datalist = testX
            print(f"Testing Data : {testX.shape}")
            
    def __getitem__(self, index):
        if self.train:
            return torch.Tensor(self.datalist[index]),self.labellist[index]
        else:
            return torch.Tensor(self.datalist[index])
    
    def __len__(self):
        return self.datalist.shape[0]
    
    def preprocess(self,img):
        '''
        For tansfer learning apply
        1. gray to BGR
        2. transform to min crop 224
        3. to tensor
        
        currently implemented in this function
        '''
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
        img = self.transform(img)
        return img
        

## Data Augmentation - Image

> invalid dim occur in images due to numpy and cv2 repr
of images



In [ ]:
# Image transformations & data augmentation

image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=256), 
        # resizes the images so the shortest side has a length of 255 pixels. 
        # The other side is scaled to maintain the aspect ratio of the image.
        
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

## Data Loading to memory

In [ ]:
# creating Data Loader

# using CustomDataSet
batch_size = 32

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
len_train_loader = len(train_dataset)

del train_dataset, test_dataset


# Datasets from folders & applying data_transformation

traindir = ''
validdir = ''

data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'valid':
    datasets.ImageFolder(root=validdir, transform=image_transforms['valid']),
}

# Dataloader iterators, make sure to shuffle

dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['valid'], batch_size=batch_size, shuffle=True)
}

# Data Visualization

In [ ]:
# make iterable object to display the cluster of images
# within the batch
images, labels = iter(train_loader).next()

# provide graid view
grid = torchvision.utils.make_grid(images)

plt.imshow(grid.numpy().transpose((1, 2, 0)))
plt.axis('off')
plt.title(labels.numpy());


# Models

## Transfer learning pretrained weights
  

1.   Loading Pretrained Weights

 press **tab** after models. to get the list
 
 *All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]*



In [ ]:
# downloading the moadel to RAM
model = models.vgg16(pretrained=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# setting training to false

for param in model.parameters():
    param.requires_grad = False

# model.classifier
# model.features


''' n_classes the number of final neurons 
(for categories -  total number of possible output)
'''

model.classifier[6] = nn.Sequential(
                      nn.Linear(4096, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))

![alt text](https://cdn-images-1.medium.com/max/1600/1*0W310-cMNHPWjErqPuGXpw.png)

## Self Customizable Convolution model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1 ,8, kernel_size=3,padding=2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 32, kernel_size=5,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=5,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

cnn = CNN()
cnn.cuda()

# 784 the number of processed neurons can be found by
# doing a trial run instead of calculation

In [ ]:
# Move to gpu
model = model.to(device)



# Distribute across 2 gpus
model = nn.DataParallel(model)

In [ ]:
# information about training parameters


total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

138,357,544 total parameters.
0 training parameters.


# Loss and Optimizer


In [ ]:
criteration = nn.NLLLoss()
optimizer = optim.Adam(model.parameters()) # auto adjustment of learning rate


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), \
                            lr=learning_rate, \
                            momentum=0.05)  #faster descent, avoid local minimum


# Training Phase

## General data training

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
  
  for i, (data, targets) in enumerate(trainloader):
    # Generate predictions
    out = model(data)
    
    # Calculate loss
    loss = criterion(out, targets)
    
    # change gradients to 0
    optimizer.zero_grad()
    
    # Backpropagation
    loss.backward()
    
    # Update model parameters
    optimizer.step()
    
    # display data
    if (i+1) % 10 == 0:
      print('Loss %.4f' %( loss.data))
      print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' %(epoch+1,
           num_epochs, i+1, len_train_loader//batch_size, loss.itme()))

### with Early stopping method
*  *to avoid overfitting of data*
* should be used preferably


In [ ]:
# Early stopping details

n_epochs_stop = 5
min_val_loss = np.Inf
epochs_no_improve = 5

checkpoint_path = "./"

# Main loop
for epoch in range(n_epochs):
  # Initialize validation loss for epoch
  val_loss = 0
  
  # Training loop
  for data, targets in trainloader:
    # Generate predictions
    out = model(data)
    loss = criterion(out, targets)
    loss.backward()
    optimizer.step()
    
  # Validation loop
  for data, targets in validloader:
    # Generate predictions 
    out = model(data)
    loss = criterion(out, targets)
    val_loss += loss

  # Average validation loss
  val_loss = val_loss / len(trainloader)

  # If the validation loss is at a minimum
  if val_loss < min_val_loss:
    # Save the model
    torch.save(model, checkpoint_path)
    epochs_no_improve = 0
    min_val_loss = val_loss

  else:
    epochs_no_improve += 1
    # Check early stopping condition
    if epochs_no_improve == n_epochs_stop:
      print('Early stopping!')

      # Load in the best model
      model = torch.load(checkpoint_path)

## image data training

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()

        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs,labels)
        
        loss.backward()
        
        optimizer.step()
        if (i+1) % 10 == 0:
          print('Loss %.4f' %( loss.data))
          print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' %(epoch+1,
               num_epochs, i+1, len_train_loader//batch_size, loss.itme()))

# Evaluating The model

## general evaluation 

In [ ]:
# Evaluating the model
model.eval()
counter = 0

# Tell torch not to calculate gradients
with torch.no_grad():
    for inputs, labels in val_loader:
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        output = model.forward(inputs)
        
        # Calculate Loss
        valloss = criterion(output, labels)
        
        # Add loss to the validation set's running loss
        val_loss += valloss.item()*inputs.size(0)

        # Since our model outputs a LogSoftmax, find the real 
        # percentages by reversing the log function
        output = torch.exp(output)
        
        # Get the top class of the output
        top_p, top_class = output.topk(1, dim=1)
        
        # See how many of the classes were correct?
        equals = top_class == labels.view(*top_class.shape)
        # Calculate the mean (get the accuracy for this batch)
        # and add it to the running accuracy for this epoch
        
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        # Print the progress of our evaluation
        counter += 1
        print(counter, "/", len(val_loader))

# Get the average loss for the entire epoch
train_loss = train_loss/len(train_loader.dataset)
valid_loss = val_loss/len(val_loader.dataset)

# Print out the information
print('Accuracy: ', accuracy/len(val_loader))
print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

## image evaluation

In [ ]:
model.eval()
ans = torch.cuda.LongTensor()

for images in test_loader:
    images = Variable(images).cuda()
    outputs = model(images)
    _,predicted = torch.max(outputs.data, 1)
    ans = torch.cat((ans,predicted),0)